# For the actual KIID Setting

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random, copy, os
import pandas as pd
random.seed(42)
np.random.seed(42)
import time
from scipy.optimize import linprog
from sklearn.metrics.pairwise import manhattan_distances
from prettytable import PrettyTable

In [2]:
%run wrapper_classes.ipynb

In [3]:
%run algorithm_functions.ipynb

# 0. Set up the experiment
## 0.1 Read in cleaned up, processed dataframe of drivers and requests

In [4]:
filtered_df = pd.read_csv('filtered data/filtered_df.csv')

In [5]:
def run_experiment_comparison(num_U,num_V,driver_quotas,rider_tolerances, alphas, betas, gammas, num_loops, verbose = False):
    driver_entries = []

    #1. First read in the drivers and riders as objects
    hack_licenses = filtered_df['hack_license'].unique()
    chosen_licenses = np.random.choice(hack_licenses,size = num_U)

    for i in range(chosen_licenses.shape[0]):
        entries = filtered_df.loc[filtered_df['hack_license'] == chosen_licenses[i]]
        driver_entries.append(entries.sample(n = 1))

    driver_entries = pd.concat(driver_entries)
    rider_entries = filtered_df.sample(n = num_V)

    drivers = []
    requests = []

    d_id = 0
    for index, row in driver_entries.iterrows():
        init_lat_bin = row['init_lat_bin']
        init_long_bin = row['init_long_bin']

        pickup_lat_bin = row['pickup_lat_bin']
        pickup_long_bin = row['pickup_long_bin']

        dropoff_lat_bin = row['dropoff_lat_bin']
        dropoff_long_bin = row['dropoff_long_bin']

        driver_race = row['driver_race']
        driver_gender = row['driver_gender']

        drivers.append(Driver(d_id = d_id, driver_race = driver_race, driver_gender = driver_gender, pickup_lat_bin = pickup_lat_bin,\
                              pickup_long_bin = pickup_long_bin, driver_init_lat = init_lat_bin, driver_init_long = init_long_bin,\
                              quota = random.choice(driver_quotas), capacity = driver_capacity))
        d_id += 1

    r_id = 0
    for index, row in rider_entries.iterrows():
        pickup_lat_bin = row['pickup_lat_bin']
        pickup_long_bin = row['pickup_long_bin']

        dropoff_lat_bin = row['dropoff_lat_bin']
        dropoff_long_bin = row['dropoff_long_bin']

        driver_race = row['driver_race']
        driver_gender = row['driver_gender']

        pickup_latitude = row['pickup_latitude']
        pickup_longitude = row['pickup_longitude']

        dropoff_latitude = row['dropoff_latitude']
        dropoff_longitude = row['dropoff_longitude']

        request_gender = row['requests_gender']
        request_race = row['requests_race']

        requests.append(Request(pickup_lat_bin = pickup_lat_bin, pickup_long_bin = pickup_long_bin, dropoff_lat_bin = dropoff_lat_bin,\
                                dropoff_long_bin = dropoff_long_bin, pickup_latitude = pickup_latitude, pickup_longitude = pickup_longitude,\
                                dropoff_latitude = dropoff_latitude, dropoff_longitude = dropoff_longitude, requests_gender = request_gender,\
                                requests_race = request_race, arrival_rate = rider_arrival_rate, distance = None, utility = None))
        r_id +=1

    set_unique_ids(drivers)
    set_unique_ids(requests)
    for r in requests:
        r.set_individual_rider_tolerance(random.choice(rider_tolerances)) #uniformly sample rider patience between {1, 2}


    #2. Set the probability matrix
    probability_matrix = np.empty([num_U,num_V])
    for row in range(num_U):
        for col in range(num_V):
            driver_race = drivers[row].race
            request_race = requests[col].race
            if driver_race == 'white' and request_race == 'white':
                probability_matrix[row,col] = 0.6
            elif driver_race == 'white' and request_race == 'black':
                probability_matrix[row,col] = 0.1
            elif driver_race == 'black' and request_race == 'white':
                probability_matrix[row,col] = 0.1
            else:
                probability_matrix[row,col] = 0.3

    #3. Construct utility matrix
    driver_pos = np.empty([num_U,2])
    rider_pos = np.empty([num_V,2])
    target_pos = np.empty([num_V,2])

    for driver_idx in range(num_U):
        driver_pos[driver_idx,0] = drivers[driver_idx].init_latitude
        driver_pos[driver_idx,1] = drivers[driver_idx].init_longitude

    for rider_idx in range(num_V):
        rider_pos[rider_idx,0] = requests[rider_idx].start_latitude
        rider_pos[rider_idx,1] = requests[rider_idx].start_longitude

        target_pos[rider_idx,0] = requests[rider_idx].end_latitude
        target_pos[rider_idx,1] = requests[rider_idx].end_longitude

    driver_rider_dis = manhattan_distances(driver_pos,rider_pos)
    trip_length = np.sum(np.abs(target_pos - rider_pos),axis = -1)
    trip_length = np.tile(trip_length,(num_U,1))

        #3.1Set the utility from three sides
    driver_utility = trip_length - driver_rider_dis
    driver_utility_matching = np.ones([num_U,num_V])
    rider_utility = -driver_rider_dis# + np.max(driver_rider_dis)
    rider_utility_matching = np.ones([num_U,num_V])
    operator_utility = trip_length


        #3.2scale them by a constant to be greater than 0
    driver_utility -= np.min(driver_utility)
    rider_utility -= np.min(rider_utility)

    if verbose:
        #print statistics of the utility matrix
        t = PrettyTable(['','mean', 'std', 'min', 'max'])
        t.title = 'utility statistics'
        t.add_row(['operator utility'] + [operator_utility.mean(),operator_utility.std(),operator_utility.min(),operator_utility.max() ])
        t.add_row(['driver utility'] + [driver_utility.mean(),driver_utility.std(),driver_utility.min(),driver_utility.max() ])
        t.add_row(['driver utility matching'] + [driver_utility_matching.mean(),driver_utility_matching.std(),driver_utility_matching.min(),driver_utility_matching.max() ])
        t.add_row(['rider utility'] + [rider_utility.mean(),rider_utility.std(),rider_utility.min(),rider_utility.max() ])
        t.add_row(['rider utility matching'] + [rider_utility_matching.mean(),rider_utility_matching.std(),rider_utility_matching.min(),rider_utility_matching.max() ])
        print(t)

    #4. solve the LPs

        #4.1 Operator LP TSGF
    bound = np.reshape(np.array([0, 1]), (1, 2))
    bounds_operator = np.tile(bound, (num_U * num_V, 1))
    profit_c = get_operator_objective_kiid_sk(operator_utility * probability_matrix)
    operator_A, operator_b = get_inequalities_operator_kiid_sk(drivers, requests, probability_matrix)
    operator_x = linprog(profit_c, operator_A, operator_b, bounds=bounds_operator, method='highs')

        #4.2 Driver LP TSGF
    driver_c = get_driver_objective_kiid_sk(driver_utility)
    driver_A, driver_b, driver_bounds = get_inequalities_driver_kiid_sk(drivers, requests, probability_matrix, driver_utility)
    driver_x = linprog(driver_c, driver_A, driver_b, bounds=driver_bounds, method='highs')

        #4.3 Rider LP TSGF
    rider_c = get_rider_objective_kiid_sk(rider_utility)
    rider_A, rider_b, rider_bounds = get_inequalities_rider_kiid_sk(drivers, requests, probability_matrix, rider_utility)
    rider_x = linprog(rider_c, rider_A, rider_b, bounds=rider_bounds, method='highs')


        #4.6 reshape the solutions
    operator_x_2d = np.reshape(operator_x.x, [num_U, num_V])
    driver_x_2d = np.reshape(driver_x.x[0:num_U * num_V], [num_U, num_V])
    rider_x_2d = np.reshape(rider_x.x[0:num_U * num_V], [num_U, num_V])


        #4.7 Calculate the statistics
    operator_operator_ub = np.sum(np.sum(operator_x_2d * operator_utility * probability_matrix, axis=-1), axis=-1)
    operator_driver_ub, operator_rider_ub = util_to_fairness(drivers, requests,
                                                             np.sum(operator_x_2d * driver_utility * probability_matrix,
                                                                    axis=1),
                                                             np.sum(operator_x_2d * rider_utility * probability_matrix,
                                                                    axis=0))

    driver_operator_ub = np.sum(np.sum(driver_x_2d * operator_utility * probability_matrix, axis=-1), axis=-1)
    driver_driver_ub, driver_rider_ub = util_to_fairness(drivers, requests,
                                                         np.sum(driver_x_2d * driver_utility * probability_matrix, axis=1),
                                                         np.sum(driver_x_2d * rider_utility * probability_matrix, axis=0))

    rider_operator_ub = np.sum(np.sum(rider_x_2d * operator_utility * probability_matrix, axis=-1), axis=-1)
    rider_driver_ub, rider_rider_ub = util_to_fairness(drivers, requests,
                                                       np.sum(rider_x_2d * driver_utility * probability_matrix, axis=1),
                                                       np.sum(rider_x_2d * rider_utility * probability_matrix, axis=0))

    operator_ubs = [operator_operator_ub, operator_driver_ub, operator_rider_ub]
    driver_ubs = [driver_operator_ub, driver_driver_ub, driver_rider_ub]
    rider_ubs = [rider_operator_ub, rider_driver_ub, rider_rider_ub]


    upper_bounds = np.array([operator_ubs[0], driver_ubs[1], rider_ubs[2]])
    if verbose:
        t = PrettyTable(['', 'profit', 'driver fairness', 'rider fairness'])
        t.title = 'LP solutions'
        t.add_row(['upper bounds'] + list(upper_bounds))
        t.add_row(['operator LP'] + list(operator_ubs))
        t.add_row(['driver LP'] + list(driver_ubs))
        t.add_row(['rider LP'] + list(rider_ubs))
        print(t)

    #5. run the experiment
        #5.1 Then run TSGF
    requests_copy = copy.deepcopy(requests)
    results_TSGF_inner = []

    for n in range(len(alphas)):
        t0 = time.time()
        alpha = alphas[n]
        beta = betas[n]
        gamma = gammas[n]

        match_results = np.zeros([num_loops,num_U,num_V])


        for i in range(num_loops):
            #TSGF
            drivers_copy = [copy.deepcopy(d) for d in drivers]
            run_TSGF_express(i,match_results,requests_copy, drivers_copy, probability_matrix, operator_x_2d,driver_x_2d, rider_x_2d,alpha=alpha, beta = beta, gamma = gamma)

        driver_mean_utility = np.mean(np.sum(match_results * driver_utility,axis = -1),axis = 0)
        rider_mean_utility = np.mean(np.sum(match_results * rider_utility,axis = -2),axis = 0)
        operator_mean_utility = np.mean(np.sum(np.sum(match_results * operator_utility,axis = -1),axis = -1),axis = 0)

        expost_d_fairness, expost_r_fairness = util_to_fairness(drivers,requests,driver_mean_utility,rider_mean_utility)
        results_TSGF_inner.append([operator_mean_utility, expost_d_fairness,expost_r_fairness])


        if verbose:
            print(alpha, beta, gamma, time.time() - t0)


    results = np.array(results_TSGF_inner)

    profit_crs = np.reshape(results[:,0] / upper_bounds[0],[alphas.shape[0],1])
    driver_fairness_crs = np.reshape(results[:,1] / upper_bounds[1],[alphas.shape[0],1])
    rider_fairness_crs = np.reshape(results[:,2] / upper_bounds[2],[alphas.shape[0],1])

    TSGF_crs = np.hstack([profit_crs, driver_fairness_crs, rider_fairness_crs])


    #run greedy algorithms
    request_copy = [copy.deepcopy(r) for r in requests]
    overall_profit = [0, 0, 0]
    driver_utilss = [[], [], []]
    rider_utilss = [[], [], []]
    for i in range(num_loops):
        random.shuffle(request_copy)

        drivers_match_o, requests_match_o = run_greedy_o(request_copy, [copy.deepcopy(d) for d in drivers],
                                                          probability_matrix, operator_utility)
        drivers_match_d, requests_match_d = run_greedy_d(request_copy, [copy.deepcopy(d) for d in drivers],
                                                          probability_matrix, driver_utility)
        drivers_match_r, requests_match_r = run_greedy_r(request_copy, [copy.deepcopy(d) for d in drivers],
                                                          probability_matrix, rider_utility)

        d_util_o, r_util_o = get_utils(drivers_match_o, requests_match_o, num_U, num_V, driver_utility, rider_utility)
        d_util_d, r_util_d = get_utils(drivers_match_d, requests_match_d, num_U, num_V, driver_utility, rider_utility)
        d_util_r, r_util_r = get_utils(drivers_match_r, requests_match_r, num_U, num_V, driver_utility, rider_utility)

        driver_utilss[0].append(d_util_o)
        rider_utilss[0].append(r_util_o)

        driver_utilss[1].append(d_util_d)
        rider_utilss[1].append(r_util_d)

        driver_utilss[2].append(d_util_r)
        rider_utilss[2].append(r_util_r)

        overall_profit[0] += get_profit(drivers_match_o, requests_match_o, operator_utility)
        overall_profit[1] += get_profit(drivers_match_d, requests_match_o, operator_utility)
        overall_profit[2] += get_profit(drivers_match_r, requests_match_o, operator_utility)

    overall_profit = np.array(overall_profit) / num_loops
    driver_utilss = np.mean(np.array(driver_utilss), axis=1)
    rider_utilss = np.mean(np.array(rider_utilss), axis=1)

    expost_d_fairness_o, expost_r_fairness_o = util_to_fairness(drivers, requests, driver_utilss[0], rider_utilss[0])
    expost_d_fairness_d, expost_r_fairness_d = util_to_fairness(drivers, requests, driver_utilss[1], rider_utilss[1])
    expost_d_fairness_r, expost_r_fairness_r = util_to_fairness(drivers, requests, driver_utilss[2], rider_utilss[2])

    greedy_o_results = np.array([overall_profit[0], expost_d_fairness_o, expost_r_fairness_o]) / upper_bounds
    greedy_d_results = np.array([overall_profit[1], expost_d_fairness_d, expost_r_fairness_d]) / upper_bounds
    greedy_r_results = np.array([overall_profit[2], expost_d_fairness_r, expost_r_fairness_r]) / upper_bounds

    greedy_crs = np.array([greedy_o_results,greedy_r_results,greedy_d_results])

    return TSGF_crs, greedy_crs


# Run the Experiment

In [6]:
start_t = time.time()

In [7]:
num_trials = 100
num_loops = 100
num_ticks = 3

In [8]:
num_U = 49
num_V = 172

driver_quotas = [3] #driver patience set to 3
rider_tolerances = [1,2]
driver_capacity = 1
rider_arrival_rate = 1

In [9]:
alphas = np.array([1,0,0])
betas = np.array([0,1,0])
gammas = np.array([0,0,1])

In [10]:
t0 = time.time()
results_TSGF = np.zeros([num_trials, num_ticks,3])
results_greedy = np.zeros([num_trials, num_ticks,3])
for i in range(num_trials):
    print(i)
    results_TSGF_single, results_greedy_single = run_experiment_comparison(num_U,num_V,driver_quotas,rider_tolerances, alphas, betas, gammas, num_loops, False)
    results_TSGF[i,:,:] =  results_TSGF_single
    results_greedy[i,:,:] = results_greedy_single
print(time.time() - t0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
2052.331593990326


In [11]:
results_TSGF_mean = np.mean(results_TSGF,axis = 0)
results_TSGF_bar = np.std(results_TSGF,axis = 0)
results_greedy_mean = np.mean(results_greedy,axis = 0)
results_greedy_bar = np.std(results_greedy,axis = 0)

In [12]:
print(results_TSGF_mean)

[[0.59459638 0.3981727  0.38428878]
 [0.51662987 0.57056257 0.4417287 ]
 [0.25201269 0.35341727 0.57370397]]


In [13]:
print(results_greedy_mean)

[[0.43130857 0.5488619  0.50336562]
 [0.37116236 0.60929982 0.56277238]
 [0.31593915 0.50371722 0.5127989 ]]
